In [1]:
from utils.setup_utils import setup
setup(globals())

W, B = load_model()
x, y = load_data()

In [ ]:
x = (x - 0.1307) / 0.3081
x = x.reshape(-1, 28 * 28)

### How to use NN weights and biases.

In [2]:

x1 = x @ W[0] + B[0]
y1 = relu(x1)

x2 = y1 @ W[1] + B[1]
y2 = relu(x2)

x3 = y2 @ W[2] + B[2]

(x3.argmax(axis=1) == y).mean()

0.9842

### Gurobipy example

In [3]:
m = gp.Model()
a = m.addVar(vtype='C', name="a")
b = m.addVar(vtype='C', name="b")
c = m.addVar(vtype='C', name="c")

m.setObjective(a + b + 2 * c, gp.GRB.MAXIMIZE)

m.addConstr(a + 2 * b + 3 * c <= 4)
m.addConstr(a + b >= 1)

m.optimize()

print(f"Optimal objective value: {m.objVal}")
print(f"Solution values: a={a.X}, b={b.X}, c={c.X}")

Academic license - for non-commercial use only - expires 2021-12-13
Using license file C:\Users\dinus\gurobi.lic


<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0xef13ba7f
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 2 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  4.000000000e+00
Optimal objective value: 4.0
Solution values: a=4.0, b=0.0, c=0.0


In [ ]:

def ginit(shape):
    return np.zeros(shape, dtype=gp.Var)

def fischetti():
    m = gp.Model()


    gurobi_x = [ginit(x.shape[1]), ginit(B[0].shape), ginit(B[1].shape), ginit(B[2].shape)]

    c = [np.random.uniform(size=x.shape[1]), np.random.uniform(size=B[0].shape), 
         np.random.uniform(size=B[1].shape), np.random.uniform(size=B[2].shape)]

    g = [np.random.uniform(size=x.shape[1]), np.random.uniform(size=B[0].shape), 
         np.random.uniform(size=B[1].shape), np.random.uniform(size=B[2].shape)]

    gurobi_s = [ginit(x.shape[1]), ginit(B[0].shape), ginit(B[1].shape), ginit(B[2].shape)]

    gurobi_z = [None, ginit(B[0].shape), ginit(B[1].shape), ginit(B[2].shape)]

    lb = [(-1 - 0.1307) / 0.3081, -50, -50, -50]
    ub = [(1 - 0.1307) / 0.3081, 50, 50, 50]


    ##### Variables #####

    for j in range(x.shape[1]):
        gurobi_x[0][j] = m.addVar(vtype='C', name=f'x_0_{j}')
        gurobi_s[0][j] = m.addVar(vtype='C', name=f's_0_{j}')

    for layer_index in range(3):
        w = W[layer_index]
        b = B[layer_index]

        n_k_prev, n_k = w.shape

    #     for i in range(n_k_prev):
    #         for j in range(n_k):
    #             W[layer_index][i, j] = m.addVar(vtype='C', name=f'w_{layer_index}_{i}_{j}')

        for j in range(n_k):
    #         B[layer_index][j] = m.addVar(vtype='C', name=f'b_{layer_index}_{j}')

            gurobi_x[layer_index + 1][j] = m.addVar(vtype='C', name=f'x_{layer_index}_{j}')
            gurobi_s[layer_index + 1][j] = m.addVar(vtype='C', name=f's_{layer_index}_{j}')

            gurobi_z[layer_index + 1][j] = m.addVar(vtype='B', name=f'z_{layer_index}_{j}')

    ##### Constraints #####

    for layer_index in range(3):
        w = W[layer_index]
        b = B[layer_index]

        n_k_prev, n_k = w.shape

        for j in range(n_k):

            _ = m.addConstr(gp.quicksum(W[layer_index][i][j] * gurobi_x[layer_index][i] 
                                     for i in range(n_k_prev)) + B[layer_index][j] ==\
                        gurobi_x[layer_index + 1][j] - gurobi_s[layer_index + 1][j]
                       )

            _ = m.addConstr(gurobi_x[layer_index + 1][j] >= 0)
            _ = m.addConstr(gurobi_s[layer_index + 1][j] >= 0)

            # https://support.gurobi.com/hc/en-us/community/posts/360077951791-if-statement-in-constraint-
            _ = m.addConstr((gurobi_z[layer_index + 1][j] == 1) >> (gurobi_x[layer_index + 1][j] <= 0))
            _ = m.addConstr((gurobi_z[layer_index + 1][j] == 0) >> (gurobi_s[layer_index + 1][j] <= 0))


            _ = m.addConstr(gurobi_x[layer_index + 1][j] >= lb[layer_index + 1])
            _ = m.addConstr(gurobi_x[layer_index + 1][j] <= ub[layer_index + 1])

            _ = m.addConstr(gurobi_s[layer_index + 1][j] >= lb[layer_index + 1])
            _ = m.addConstr(gurobi_s[layer_index + 1][j] <= ub[layer_index + 1])


    for j in range(x.shape[1]):        
        _ = m.addConstr(gurobi_x[0][j] >= lb[0])
        _ = m.addConstr(gurobi_x[0][j] <= ub[0])        
        _ = m.addConstr(gurobi_s[0][j] >= lb[0])
        _ = m.addConstr(gurobi_s[0][j] <= ub[0])


    ##### Objective #####

    s1 = m.addVar(vtype='C', name=f's1')
    s2 = m.addVar(vtype='C', name=f's1')

    _ = m.addConstr(s1 == gp.quicksum(
        gp.quicksum(
            gurobi_x[layer_index][j] * c[layer_index][j]
            for j in range(gurobi_x[layer_index].shape[0])) 
         for layer_index in range(4)))

    _ = m.addConstr(s2 == gp.quicksum(
        gp.quicksum(
            gurobi_z[layer_index + 1][j] * g[layer_index + 1][j] 
            for j in range(gurobi_x[layer_index + 1].shape[0])) 
         for layer_index in range(3)))

    m.setObjective(s1 + s2, gp.GRB.MINIMIZE)

    m.optimize()
    
    return gurobi_x

fischetti_x = fischetti()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 6792 rows, 3136 columns and 277942 nonzeros
Model fingerprint: 0xcd659786
Model has 1044 general constraints
Variable types: 2614 continuous, 522 integer (522 binary)
Coefficient statistics:
  Matrix range     [3e-08, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e-05, 5e+01]
Presolve removed 5226 rows and 786 columns
Presolve time: 0.79s
Presolved: 1566 rows, 2350 columns, 271932 nonzeros
Variable types: 1828 continuous, 522 integer (522 binary)

Root relaxation: objective 3.250248e+00, 1384 iterations, 0.28 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.25025    0  272          -    3.25025      -     -    1s
     0     0    3.34869    0  268          -   

  4786  3361   16.22357  218   55   17.82398    4.19283  76.5%   185  511s
H 4965  3114                      17.4004518    4.19283  75.9%   185  608s
  4969  3175   17.19038  263   34   17.40045    4.60218  73.6%   185  624s
  5048  3311    6.43194   26  231   17.40045    4.60218  73.6%   185  636s
  5216  3385   14.50087   46  217   17.40045    4.60218  73.6%   187  648s
  5350  3515   16.92532   59  196   17.40045    4.70312  73.0%   188  662s
  5545  3686    6.08548   36  215   17.40045    4.70312  73.0%   188  675s
  5716  3883    8.41783   76  174   17.40045    4.70312  73.0%   189  690s
  5913  4069   11.06741  130  130   17.40045    4.70312  73.0%   189  705s
  6130  4307   14.58753  181   86   17.40045    4.72918  72.8%   188  721s
  6394  4526    6.55150   42  229   17.40045    4.83852  72.2%   187  737s
  6613  4749    9.84187  101  156   17.40045    4.83852  72.2%   189  754s
  6836  4945   13.72014  159  101   17.40045    4.83852  72.2%   189  772s
  7062  5207   17.19827  

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Software\envs\env_1\lib\site-packages\ipykernel\iostream.py", line 490, in write
    def write(self, string: str) -> int:
KeyboardInterrupt: 


 33713 29086    7.68429   59  198   16.81003    5.47676  67.4%   204 2611s
 34108 29422   13.02236  145  117   16.81003    5.49091  67.3%   204 2640s
 34483 29796    6.68221   52  218   16.81003    5.49091  67.3%   204 2669s
